In [1]:
# pip install keras2onnx

In [2]:
import os 
from glob import glob

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline

In [3]:
train_dir = 'mnist_png/mnist_png/training/'
test_dir = 'mnist_png/mnist_png/testing/'

In [4]:
batch_size = 32
num_classes = 10
epochs = 1

learning_rate = 0.001

In [5]:
def build_model(input_shape):
    inputs = layers.Input(input_shape)
    net = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(32, (3, 3))(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPooling2D(pool_size=(2, 2))(net)
    net = layers.Dropout(0.25)(net)
    
    net = layers.Conv2D(64, (3, 3), padding='same')(net)
    net = layers.Activation('relu')(net)
    net = layers.Conv2D(64, (3, 3))(net)
    net = layers.Activation('relu')(net)
    net = layers.MaxPooling2D(pool_size=(2, 2))(net)
    net = layers.Dropout(0.25)(net)

    net = layers.Flatten()(net)
    net = layers.Dense(512)(net)
    net = layers.Activation('relu')(net)
    net = layers.Dropout(0.5)(net)
    net = layers.Dense(num_classes)(net)
    net = layers.Activation('softmax')(net)

    return tf.keras.Model(inputs=inputs, outputs=net)

model = build_model((28, 28, 1))
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate),
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
#     save_to_dir='out_images'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(28, 28),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical')

Found 60000 images belonging to 10 classes.
Found 10000 images belonging to 10 classes.


## Callbacks

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='my_model.h5', # 저장
        monitor='val_loss',
        save_best_only=True, # 가장 좋은 모델
    ),
    keras.callbacks.ReduceLRONPlateau(
        monitor='val_acc',
        factor=0.1, # 콜백이 호출되면 학습률을 10배로 줄임
        patience=10,
    ),
    # `val_loss`가 2번의 에포크에 걸쳐 향상되지 않으면 훈련을 멈춥니다.
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
    # `./logs` 디렉토리에 텐서보드 로그를 기록니다.
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]

## Custom Callbacks

https://subinium.github.io/Keras-7/

In [7]:

model.fit_generator(train_generator,
                    steps_per_epoch=2000,
                    epochs=1,
                    callbacks=callbacks
                   )

model.evaluate(validation_data=validation_generator,
               validation_steps=800
              )

Instructions for updating:
Use tf.cast instead.
  17/1875 [..............................] - ETA: 10:22 - loss: 2.2512 - acc: 0.1599

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-a55188c499b0>", line 6, in <module>
    validation_steps=800
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1426, in fit_generator
    initial_epoch=initial_epoch)
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_generator.py", line 191, in model_iteration
    batch_outs = batch_function(*batch_data)
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py", line 1191, in train_on_batch
    outputs = self._fit_function(ins)  # pylint: disable=not-callable
  File "C:\Users\June\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\

KeyboardInterrupt: 